# Running Computational Alchemy for Catalysis: Transition State Barrier Height Predictions
This notebook outlines Python functions that use information from VASP calculations to predict descriptors for hypothetical catalyst. These functions utilize tools from the Atomic Simulation Environment (ASE) to manipulate catalyst surface models and perform calculations with computational alchemy. 

In this example, we apply computational alchemy on reference set of nudged elastic band (NEB) calculcations of CH4 dehydrogenation on Pt(111) to predict transition state barrier heights of the same process on hypothetical alloys of Pt(111). 

In [1]:
import comp_alchemy

In [2]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CustomJS, Range1d
from bokeh.models.annotations import Slope
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

This example uses an NEB calculation with 10 images in total. Below we plot the total energy for each image. You also can hover your mouse over any data point to visualize the image

In [3]:
images = range(10)
image_energies = pd.read_csv('reference_images_energies.csv')
image_energies = image_energies['energy']

In [4]:
output_notebook()

en_source = ColumnDataSource(
data=dict(
x=[int(im) for im in images],
y=image_energies,
ads_imgs=['neb_images/'+str(im)+'/'+'neb_image_'+str(im)+'.png' for im in images]))

en_tooltips="""
            <p>
                <span style="font-size: 25px;">NEB Image </span>
                <span style="font-size: 25px;">$index</span>
            </p>
            <p>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
"""

ef = figure(plot_width=900, plot_height=600,y_range=(min(image_energies)-0.5,max(image_energies)+0.5),tooltips=en_tooltips)

ef.xaxis.axis_label = "NEB Image Number"
ef.xaxis.axis_label_text_font_style = "bold"
ef.xaxis.axis_label_text_font_size = "12pt"
ef.yaxis.axis_label = "Energy (eV)"
ef.yaxis.axis_label_text_font_style = "bold"
ef.yaxis.axis_label_text_font_size = "12pt"
ef.title.text_font_size = '12pt'
ef.title.align = 'center'

ef.circle('x', 'y', size=45, source=en_source)
ef.line('x','y', line_dash="dashed", source=en_source)

show(ef, notebook_handle=True)

Loading BokehJS ...

From the plot above, it's apparent that image 5 is the transition state for CH4 dehydrogenation.

Next we make ASE Atoms objects with VASP POSCARs and CONTCARs for the bare catalyst surface (slab) and the catalyst surface with the adsorbate (ads) for each image. These models and the results of the VASP calculations for each will be used as our reference for computational alchemy calculations.

In [29]:
from ase.io import read 
from ase.visualize import view
slab = read('slab/POSCAR',format='vasp')
slab_contcar = read('slab/CONTCAR',format='vasp')
ads = [read(f'neb_images/{str(im)}/POSCAR',format='vasp') for im in images]
ads_contcar = [read(f'neb_images/{str(im)}/CONTCAR',format='vasp') for im in images]

In [30]:
from comp_alchemy.ads_slab_pairs import pairs
p = [pairs(slab,slab) for a in ads]

### Working with electrostatic potentials in VASP OUTCAR
Next we can grab the atom-centered electrostatic potentials (ESPs) printed at the end of the OUTCAR with `grab_esp()`. 
Here we grab ESPs for slab and ads and calculate the differences with `espdiff()`. The ESP differnces define the alchemical derivatives that facilitate predictions with alchemy.

In [31]:
from comp_alchemy.elec_stat_pot import (grab_esp,espdiff,remove_duplicate_espdiffs,heatmap)
slab_elec = grab_esp(poscar='slab/POSCAR',outcar='slab/trunc-OUTCAR')
ads_elec = [grab_esp(poscar=f'neb_images/{str(im)}/POSCAR',
                     outcar=f'neb_images/{str(im)}/trunc-OUTCAR') for im in images]
diffs = [espdiff(elec1=slab_elec,elec2=ads_elec[im],pair=p[im]) for im in images]

In [32]:
diffs0 = [round(d,4) for d in diffs[0]]
print(f'''
Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
{slab_elec}

Ads:
{ads_elec[0]}

Differences in electrostatic potentials between ads and slab
{diffs0}''')


Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
[-73.7064, -73.7064, -73.7064, -73.7064, -74.1493, -74.1493, -74.1493, -74.1493, -74.0957, -74.0957, -74.0957, -74.0957, -73.6192, -73.6192, -73.6192, -73.6192]

Ads:
[-74.1863, -74.186, -74.1864, -74.1866, -74.6565, -74.6556, -74.6567, -74.6575, -74.6527, -74.6519, -74.6519, -74.6528, -74.1955, -74.1747, -74.1757, -74.1765, -53.2355, -36.6325, -36.6332, -36.6324, -36.6331]

Differences in electrostatic potentials between ads and slab
[-0.4799, -0.4796, -0.48, -0.4802, -0.5072, -0.5063, -0.5074, -0.5082, -0.557, -0.5562, -0.5562, -0.5571, -0.5763, -0.5555, -0.5565, -0.5573]


### Visualizing alchemical derivatives
With the `heatmap()` function, visualize the alchemical derivatives with ASE GUI by running the cell below, pressing "c" in ASE GUI, and under "Choose how the atoms are colored:" selecting "By initial charge" 

In [33]:
slab_vis = [heatmap(poscar='slab/POSCAR',dexlist=[dex[0] for dex in p[im]],espdiffs=diffs[im]) for im in range(0,len(images))]
#view(slab_vis)

### Creating hypothetical materials by making transmutations
Below we use the `index_transmuted()` function to grab indexes of atoms that we want to transmute in our reference slabs. We specify the number of transmutable atoms by setting `transmute_num`. 

This function also grabs indexes of atoms at the bottom of the slab that we can counter-transmute to maintain isoelectronicity. We set the number of counter-transmutable atoms with `counter_num`.

In [34]:
from ase import Atom
from comp_alchemy.alloy_index import index_transmuted, transmuter, transmuted_directory_names
metal = Atom('Pt')
[transmute, counter] = index_transmuted(slab=slab,
                                        transmute_atom_sym=metal.symbol,
                                        counter_atom_sym=metal.symbol,
                                        transmute_num=8,
                                        counter_num=2,
                                        symmetric = False)

transmute = [transmute for im in images]
counter = [counter for im in images]

In [35]:
print(f'''Indices of atoms to be transmuted in the top two surface layers.
{transmute[0]}

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
{counter[0]}

There are {len(transmute)} sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of {len(counter)} sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of {len(transmute)*len(counter)} hypothetical configurations of alloys.
''')

Indices of atoms to be transmuted in the top two surface layers.
[12, 13, 14, 15, 8, 9, 10, 11]

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
[0, 1]

There are 10 sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of 10 sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of 100 hypothetical configurations of alloys.



In this example, we transmute surface Pt atoms to Au or Ir (change in nuclear charge of +/- 1) and counter-transmute Pt atoms in the bottom layers to Ir or Au (change in nuclear charge of -/+ 1). Below we construct an array with Au and Ir Atom objects to use in the `transmuter()` function.

In [36]:
charge = [1,-1]
chargelen = len(charge)
all_transmute = []
for c in charge:
    transmute_atom = Atom(metal.symbol)
    transmute_atom.number += c
    counter_transmute_atom = Atom(metal.symbol)
    counter_transmute_atom.number -= c
    all_transmute.append([counter_transmute_atom,transmute_atom])

Next, we make transmutations to Atoms objects of slab and ads from their respective CONTCARs and give labels for each new model with the `transmuted_directory_names()` function. Now we can visualize these new systems and write POSCARs to do VASP calculations for benchmarking.

In [37]:
print('Transmuted Slab Labels')
transmuted_slabs = []
dir_slab = []
for k in all_transmute:
    slab_single_image = []
    slabdir_single_image = []
    for i,c in enumerate(counter[0]):
        for j,t in enumerate(transmute[0]):
            slab_single_image.append(transmuter(slab=slab_contcar,
                                           atomdex=[c,t],
                                           trans=k))
            slab_dname = transmuted_directory_names(bdex=i,
                                           tdex=j,
                                           dexes=[c,t],
                                           atoms_array=k)
            slabdir_single_image.append(slab_dname)
            print(slab_dname)
    transmuted_slabs.append(slab_single_image)
    dir_slab.append(slabdir_single_image)

Transmuted Slab Labels
0.0.Ir0.Au12
0.1.Ir0.Au13
0.2.Ir0.Au14
0.3.Ir0.Au15
0.4.Ir0.Au8
0.5.Ir0.Au9
0.6.Ir0.Au10
0.7.Ir0.Au11
1.0.Ir1.Au12
1.1.Ir1.Au13
1.2.Ir1.Au14
1.3.Ir1.Au15
1.4.Ir1.Au8
1.5.Ir1.Au9
1.6.Ir1.Au10
1.7.Ir1.Au11
0.0.Au0.Ir12
0.1.Au0.Ir13
0.2.Au0.Ir14
0.3.Au0.Ir15
0.4.Au0.Ir8
0.5.Au0.Ir9
0.6.Au0.Ir10
0.7.Au0.Ir11
1.0.Au1.Ir12
1.1.Au1.Ir13
1.2.Au1.Ir14
1.3.Au1.Ir15
1.4.Au1.Ir8
1.5.Au1.Ir9
1.6.Au1.Ir10
1.7.Au1.Ir11


In [38]:
transmuted_ads = []
dir_ads = []
for im in range(0,len(images)):
    ads_single_image = []
    adsdir_single_image = []
    for k in all_transmute:
        ads_single = []
        adsdir_single = []
        for i,c in enumerate(counter[im]):
            for j,t in enumerate(transmute[im]):
                ads_single.append(transmuter(slab=ads_contcar[im],
                                                     atomdex=[c,t],
                                                     trans=k))
                adsdir_single.append(transmuted_directory_names(bdex=i,
                                                              tdex=j,
                                                              dexes=[c,t],
                                                              atoms_array=k))
        ads_single_image.append(ads_single)
        adsdir_single_image.append(adsdir_single)
    transmuted_ads.append(ads_single_image)
    dir_ads.append(adsdir_single_image)

### Calculating binding energy (BE) with computational alchemy
For validation purposes, we could then setup directories to run VASP calculations on our transmuted surface models. It's most convinient to analyze these calculations if they are each stored in a directory named with the labels generated above. 
Once all VASP calculations are complete, we can calculate binding energies on our transmuted surfaces using alchemy, then compare against DFT binding energies.

First we calculate the BEs of our reference systems, each NEB image.

In [39]:
from comp_alchemy.read_oszicar import grab_energy
adsorbate_energy = -221.81
slab_energy = grab_energy(oszicar='slab/trunc-OSZICAR')
ads_energy = image_energies
ref_be = [slab_energy + adsorbate_energy - ae for ae in ads_energy]

The DFT-calculated energies for the transmuted systems are stored in `transmuted_ads_energies.csv` and `transmuted_slab_energies.csv`. Below we read the data into dataframes and extract the energy values. 

In [40]:
transmuted_slab_data = pd.read_csv('transmuted_slab_energies.csv',index_col='labels')
transmuted_slab_energies = [[transmuted_slab_data['energies'][lab] for lab in dir_slab[i]] for i in range(chargelen)]

In [41]:
transmuted_ads_data = pd.read_csv('transmuted_ads_energies.csv',index_col=['labels','image','deltaZ'])
transmuted_ads_energies = [[[transmuted_ads_data['energies'][lab,im,charge[i]] for lab in dir_ads[im][i]] for i in range(chargelen)] for im in images]

Next, we calculate BEs for the transmuted variations of each NEB image. With those and the reference BEs, we then calculate the $\Delta$BEs. 

In [42]:
transmuted_be = [[[transmuted_slab_energies[i][j] + adsorbate_energy - transmuted_ads_energies[im][i][j] for j in range(len(transmuted_slab_energies[i]))] for i in range(chargelen)] for im in images]
dft_del_be = [[[transmuted_be[im][i][j] - ref_be[im] for j in range(len(transmuted_be[im][i]))]for i in range(chargelen)] for im in images]

Finally, we can use the `alc_be()` function to take the dot product of an array of charge differences and an array of the ESP differences to approximate the $\Delta$BEs. Below we've used this function to construct the charge difference array and calculate BE for the transmuted systems of each NEB image. These arrays contain **all necessary information to predict BEs.**

In [43]:
from comp_alchemy.binding_energy import alc_be
dn = [[[[alc_be(transmute=[transmute[im][k]],
               counter=[counter[im][j]],
               espdiffs=diffs[im],
               charge=charge[i])[0] for k in range(len(transmute[im]))] for j in range(len(counter[im]))] for i in range(chargelen)] for im in images]
alc_del_be = [[[[alc_be(transmute=[transmute[im][k]],
               counter=[counter[im][j]],
               espdiffs=diffs[im],
               charge=charge[i])[1] for k in range(len(transmute[im]))] for j in range(len(counter[im]))] for i in range(chargelen)] for im in images]

alc_del_be = [[[item for sublist in alc_del_be[im][i] for item in sublist] for i in range(chargelen)] for im in images]

### Benchmarking computational alchemy against DFT

Now that the BEs are calculated, we can compute compute reaction barriers and evaluate the accuracy of computational alchemy against DFT. For each BE, we calculate the absolute error between alchemy and DFT below.

In [44]:
from numpy import average
ae = [[[abs(alc_del_be[im][i][j] - dft_del_be[im][i][j]) for j in range(len(alc_del_be[im][i]))] for i in range(chargelen)] for im in images]
mae = [round(average(ae[im]),3) for im in range(len(images))]

Below we also calcuate the distance between each transmute site and the binding site of CH4. This descriptor will help us analyze the data.

In [45]:
adsorbate_site = 'C'
adsorbate_indexes = [a.index for a in ads_contcar[0] if a.symbol == adsorbate_site]
adsorbate_positions = [[ads_contcar[0][d].position[0],ads_contcar[0][d].position[1],ads_contcar[0][d].position[2]]for d in adsorbate_indexes]
adsorbate_positions = [items for item in adsorbate_positions for items in item if item[2] == max([pos[2] for pos in adsorbate_positions])]
distances = [[np.sqrt((ads_contcar[0][t].position[0]-adsorbate_positions[0])**2+(ads_contcar[0][t].position[1]-adsorbate_positions[1])**2+(ads_contcar[0][t].position[2]-adsorbate_positions[2])**2) for t in transmute[0]*len(counter[0])] for i in range(chargelen)]

In [46]:
output_notebook()
im = 3

pf = figure(plot_width=800, plot_height=800,title="Parity Plot for BE Predictions on NEB Images with Computational Alchemy")

parity = Slope(gradient=1,y_intercept=0,)
pf.add_layout(parity)

colors = ['blue','green','red','yellow','magenta','cyan','indigo']
pf.x_range=Range1d(-1,1)
pf.y_range=Range1d(-1,1)

r = []
for i in range(chargelen):
    source = ColumnDataSource(data=dict(
                x=[d for d in dft_del_be[im][i]],
                y=[a for a in alc_del_be[im][i]],
                rad=[d/50 for d in distances[i]],
                color=[colors[i]]*len(alc_del_be[im][i])))
    r.append(pf.circle('x', 'y', radius='rad', fill_color ='color',source=source, line_color="black", line_width=2, fill_alpha=0.5, legend='delZ = '+str(charge[i])))

def update(neb_image):
    neb_image = int(neb_image)
    for i in range(chargelen):
        r[i].data_source.data['x'] = dft_del_be[neb_image][i]
        r[i].data_source.data['y'] = alc_del_be[neb_image][i]
    push_notebook()

Loading BokehJS ...

In [47]:
show(pf, notebook_handle=True)

In [48]:
interact(update, neb_image=[int(n) for n in np.linspace(0,9,10)])

interactive(children=(Dropdown(description='neb_image', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Outp…

<function __main__.update(neb_image)>

In [49]:
dft_ea = [[dft_del_be[5][i][j] - dft_del_be[0][i][j] for j in range(len(dft_del_be[0][i]))] for i in range(chargelen)]
alc_ea = [[alc_del_be[5][i][j]-alc_del_be[0][i][j] for j in range(len(alc_del_be[0][i]))] for i in range(chargelen)]

In [50]:
ea_abs_error = [[abs(alc_ea[i][j] - dft_ea[i][j]) for j in range(len(alc_ea[i]))] for i in range(chargelen)]
ea_mean_abs_error = round(np.average(ea_abs_error),3)

In [51]:
for i in range(chargelen):
    for j in range(len(dft_ea[i])):
        print(dir_slab[i][j],dft_ea[i][j],alc_ea[i][j],ea_abs_error[i][j])

0.0.Ir0.Au12 -0.8077800000000082 -0.7267000000000081 0.08108000000000004
0.1.Ir0.Au13 -0.00836000000001036 -0.05310000000000059 0.04473999999999023
0.2.Ir0.Au14 -0.048570000000012215 -0.011499999999998067 0.03707000000001415
0.3.Ir0.Au15 0.007530000000002701 -0.033699999999996066 0.04122999999999877
0.4.Ir0.Au8 0.01099999999999568 0.03649999999998954 0.02549999999999386
0.5.Ir0.Au9 0.010919999999998709 0.011199999999988108 0.00027999999998939984
0.6.Ir0.Au10 -0.057390000000026475 -0.00940000000001362 0.047990000000012856
0.7.Ir0.Au11 -0.02529000000004089 -0.006100000000003547 0.019190000000037344
1.0.Ir1.Au12 -0.822829999999982 -0.7314000000000078 0.09142999999997414
1.1.Ir1.Au13 -0.014130000000022847 -0.057800000000000296 0.04366999999997745
1.2.Ir1.Au14 -0.05345000000002642 -0.016199999999997772 0.03725000000002865
1.3.Ir1.Au15 0.00354999999998995 -0.03839999999999577 0.04194999999998572
1.4.Ir1.Au8 0.017269999999996344 0.031799999999989836 0.014529999999993493
1.5.Ir1.Au9 0.00151999

In [69]:
output_notebook()

pf2 = figure(plot_width=800, plot_height=800, title="Parity Plot for Barrier Height Predictions with Computational Alchemy")


pf2.xaxis.axis_label = "DFT"
pf2.xaxis.axis_label_text_font_style = "bold"
pf2.xaxis.axis_label_text_font_size = "18pt"
pf2.xaxis.major_label_text_font_size = "14pt"
pf2.xaxis.major_label_text_font_style = "bold"
pf2.xaxis.major_tick_line_width = 2

pf2.yaxis.axis_label = "Alchemy"
pf2.yaxis.axis_label_text_font_style = "bold"
pf2.yaxis.axis_label_text_font_size = "18pt"
pf2.yaxis.major_label_text_font_size = "14pt"
pf2.yaxis.major_label_text_font_style = "bold"
pf2.yaxis.major_tick_line_width = 2

pf2.title.text_font_size = '12pt'
pf2.title.align = 'center'

parity = Slope(gradient=1,y_intercept=0,)
pf2.add_layout(parity)

colors = ['blue','green','red','yellow','magenta','cyan','indigo']
mx = []
mn =[]
for i in range(chargelen):
    mx.append(max(dft_ea[i]+alc_ea[i]))
    mn.append(min(dft_ea[i]+alc_ea[i]))
    source2 = ColumnDataSource(data=dict(
            xx=[d for d in dft_ea[i]],
            yy=[a for a in alc_ea[i]],
            rad=[d/50 for d in distances[i]],
            color=[colors[i]]*len(alc_ea[i])))
    pf2.circle('xx', 'yy', radius='rad', fill_color ='color',source=source2, line_color="black", line_width=2, fill_alpha=0.5, legend='delZ = '+str(charge[i]))

pf2.asterisk(0,0,size=50,line_width=5,color='red',legend='Ea_ref')

#pf2.x_range=Range1d(min(mn)*0.95,max(mx)*1.05)
#pf2.y_range=Range1d(min(mn)*0.95,max(mx)*1.05)
pf2.x_range=Range1d(-1,1)
pf2.y_range=Range1d(-1,1)

pf2.legend.location = "top_left"
pf2.legend.click_policy = "hide"
pf2.legend.label_text_font_size = "18pt"
pf2.legend.glyph_height = 65
pf2.legend.glyph_width = 65
show(pf2)

Loading BokehJS ...

In [162]:
datsum = pd.DataFrame(columns=['labels','DFT Binding Energy','Alchemy Binding Energy',
                               'BE Absolute Errors','Transmutation Site Index',
                              'ESP Difference at Transmute Site','DFT Activation Energy','Alchemy Activation Energy','Ea Absolute Errors'])
for i in images:
    datsum = datsum.append({'labels': str(int(i))},ignore_index=True)
    for j in range(chargelen):
        for k in range(len(dir_ads[i][j])):
            datsum = datsum.append({'labels': dir_ads[i][j][k],
                                    'DFT Binding Energy': dft_del_be[i][j][k],
                                    'Alchemy Binding Energy': alc_del_be[i][j][k],
                                    'BE Absolute Errors': ae[i][j][k],
                                    'Transmutation Site Index': (transmute[i]*len(counter))[k],
                                    'ESP Difference at Transmute Site': ([diffs[i][l] for l in transmute[i]]*len(counter))[k],
                                    'DFT Activation Energy': dft_ea[j][k],
                                    'Alchemy Activation Energy': alc_ea[j][k],
                                    'Ea Absolute Errors': ea_abs_error[j][k]
                                    }, ignore_index=True)
print(datsum)

labels  DFT Binding Energy  Alchemy Binding Energy  \
0               0                 NaN                     NaN   
1    0.0.Ir0.Au12           -0.063566                 -0.0964   
2    0.1.Ir0.Au13           -0.048324                 -0.0756   
3    0.2.Ir0.Au14           -0.052155                 -0.0766   
4    0.3.Ir0.Au15           -0.049953                 -0.0774   
5     0.4.Ir0.Au8           -0.046856                 -0.0771   
6     0.5.Ir0.Au9           -0.045077                 -0.0763   
7    0.6.Ir0.Au10           -0.047016                 -0.0763   
8    0.7.Ir0.Au11           -0.046769                 -0.0772   
9    1.0.Ir1.Au12           -0.064623                 -0.0967   
10   1.1.Ir1.Au13           -0.047496                 -0.0759   
11   1.2.Ir1.Au14           -0.052414                 -0.0769   
12   1.3.Ir1.Au15           -0.050053                 -0.0777   
13    1.4.Ir1.Au8           -0.046606                 -0.0774   
14    1.5.Ir1.Au9           -0.04628